# Combining msaexp and PypeIt reduction

In [ ]:
import os, sys
import numpy as np

os.environ["CRDS_PATH"] = "/Users/myue/Research/Projects/JWST/dependencies/crds_cache/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

from astropy.io import fits

In [ ]:
DO_NOT_USE = 4

In [ ]:
import os
import glob
import yaml
import warnings
import time

import numpy as np
import matplotlib.pyplot as plt

import grizli
from grizli import utils, jwst_utils
jwst_utils.set_quiet_logging()
utils.set_warnings()

import astropy.io.fits as pyfits
import jwst.datamodels
import jwst

import mastquery.jwst

import msaexp
from msaexp import pipeline
import msaexp.slit_combine

print(f'jwst version = {jwst.__version__}')
print(f'grizli version = {grizli.__version__}')
print(f'msaexp version = {msaexp.__version__}')

plt.rcParams['scatter.marker'] = '.'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['grid.linestyle'] = ':'

In [ ]:
from copy import deepcopy

def safe_replace_slice(orig_arr, new_arr, slitslice):
    copy_arr = deepcopy(orig_arr)
    copy_arr[slitslice] = new_arr
    return copy_arr

# Reduce the spectra with msaexp

In [ ]:
group_kws = dict(
    diffs=True, # For nod differences
    #undo_barshadow=2, # For msaexp barshadow correction
    min_bar=0.35, # minimum allowed value for the (inverse) bar shadow correction
    position_key="y_index",
    trace_with_ypos=True, # Include expected y shutter offset in the trace
    trace_from_yoffset=True,
    #flag_profile_kwargs=None, # Turn off profile flag
    #bad_shutter_names=[-1]
)

flag_profile_kwargs = dict(require_multiple=True, make_plot=True, grow=2, nfilt=-32)

group_kws['diffs'] = True
group_kws['flag_profile_kwargs'] = flag_profile_kwargs

In [ ]:
all_reduced_2d = glob.glob('jw04713*_phot.*.4713_*.fits')

In [ ]:
all_objid = []
for f in all_reduced_2d:
    try:
        objid = int(f.split('.')[-2].split('_')[-1])
    except ValueError:
        continue
    if not objid in all_objid:
        all_objid.append(objid)

In [ ]:
!mkdir xobj_saves

In [ ]:
import pickle

In [ ]:
def save_xobj(xobj, outdir):
    for key in xobj.keys():
        slits = xobj[key]['obj'].slits
        for index in range(len(xobj[key]['obj'].files)):
            filename = xobj[key]['obj'].files[index]

            shape = xobj[key]['obj'].shapes[index]
            sh = xobj[key]['obj'].sh
            hotpix = xobj[key]['obj'].flagged_hot_pixels[index][0].reshape(shape)

            # other data need special treatment
            slit = slits[index]

            data = safe_replace_slice(slit.data, \
                                      xobj[key]['obj'].data[index].reshape(sh),\
                                     slit.slice)
            dq = safe_replace_slice(slit.dq, \
                                    xobj[key]['obj'].dq[index].reshape(sh),\
                                     slit.slice)
            var_poisson = safe_replace_slice(slit.var_poisson, \
                                             xobj[key]['obj'].var_poisson[index].reshape(sh),\
                                     slit.slice)
            var_rnoise = safe_replace_slice(slit.var_rnoise, \
                                            xobj[key]['obj'].var_rnoise[index].reshape(sh),\
                                     slit.slice)
            
            hotpix[hotpix>0] = DO_NOT_USE
            dqall = np.bitwise_or(dq, hotpix)
            
            # save it
            outfile = filename.replace('.fits', 'tmp.fits')
            hdu0 = fits.PrimaryHDU()
            #SCI, DQ, ERR, WAVELENGTH, BARSHADOW, VAR_POISSON, VAR_RNOISE, VAR_FLAT, PATHLOSS_PS, PATHLOSS_UN
            hdu1 = fits.ImageHDU(data=data, name='DATA')
            hdu2 = fits.ImageHDU(data=dqall, name='DQ')
            hdu3 = fits.ImageHDU(data=var_poisson, name='VAR_POISSON')
            hdu4 = fits.ImageHDU(data=var_rnoise, name='VAR_RNOISE')
            #hdu5 = fits.ImageHDU(data=WAVELENGTH, name='WAVELENGTH')

            hdulist = fits.HDUList([hdu0, hdu1, hdu2, hdu3, hdu4])
            
            hdulist.writeto(outdir+'/'+outfile, overwrite=True)

In [ ]:
DRIZZLE_KWS = dict(
    step=1,
    with_pathloss=True,
    wave_sample=1.05,
    ny=13,
    dkws=dict(oversample=16, pixfrac=0.8),
)

for objid in all_objid:
    
    files = glob.glob(f'jw*_phot*{objid}.fits')
    files.sort()
    print(files)
    
    outroot = 'original%d'%objid
    hdul, xobj = msaexp.slit_combine.extract_spectra(
                target=str(objid),
                root=outroot,
                **group_kws,
                get_xobj=True,
                files=files,
                drizzle_kws=DRIZZLE_KWS,
                undo_barshadow=2
                )

    for key in xobj.keys():
        save_xobj(xobj, './xobj_saves/')

# we can now update the cal files, so that PypeIt can read them properly

In [ ]:


def update_cal_file(calfile, msaexp_out_dir):
    # make a backup
    calfile_old = calfile.replace('cal', 'calold')
    calfile_new = calfile.replace('cal', 'calnew')
    os.system('cp %s %s'%(calfile, calfile_old))

    hdulist = fits.open(calfile)
    for index in range(1, len(hdulist)):
        if hdulist[index].header['EXTNAME']=='SCI':
            #print(hdulist[index].header.keys)
            srcname = hdulist[index].header['SRCNAME']
            sltid = hdulist[index].header['SLITID']
            #print(srcname, sltid)
            
            msaexp_tmpfile_root = os.path.basename(calfile).replace('cal.fits', 'phot')+\
                                '.%03d.'%hdulist[index].header['SLITID  ']+\
                                srcname+'tmp.fits'
            msaexp_tmpfile = os.path.join(msaexp_out_dir, msaexp_tmpfile_root)
            if not os.path.exists(msaexp_tmpfile):
                #print(msaexp_tmpfile_root)
                continue

            else:
                msaexp_hdus = fits.open(msaexp_tmpfile)
                hdulist[index].data = msaexp_hdus[1].data
                hdulist[index+1].data = msaexp_hdus[2].data
                hdulist[index+5].data = msaexp_hdus[3].data
                hdulist[index+6].data = msaexp_hdus[4].data

        else:
            continue

    hdulist.writeto(calfile_new, overwrite=True)
    hdulist.close()

In [ ]:
calfiles = glob.glob('./pypeit/jw04713*_cal.fits')
calfiles.sort()
#len(calfiles)

for f in calfiles:
    update_cal_file(f, './xobj_saves/')

# OK, now you should be able to run PypeIt reduction for the files (please go to step 3).